# Tratamiento de datos missing

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En concreto, en muchos conjuntos de datos interesantes faltan algunos datos.
Para complicar aún más las cosas, las distintas fuentes de datos pueden indicar la falta de datos de formas diferentes.

En este notebook, discutiremos algunas consideraciones generales para los datos que faltan, discutiremos cómo Pandas elige representarlos, y demostraremos algunas herramientas incorporadas de Pandas para manejar los datos que faltan en Python.
Aquí y a lo largo del libro, nos referiremos a los datos perdidos en general como valores *None*, *NaN*, *NA*, *NaT*...

In [18]:
# None (str)
# NaN (float)
# NA (entero)
# NaT (tiempo)

## Compromisos en las convenciones sobre datos missing

Se han desarrollado varios esquemas para indicar la presencia de datos que faltan en una tabla o DataFrame.
En general, giran en torno a una de estas dos estrategias: utilizar una *máscara* que indique globalmente los valores que faltan, o elegir un *valor centinela* que indique una entrada que falta.

En el enfoque de enmascaramiento, la máscara puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque de centinela, el valor centinela podría ser alguna convención específica de los datos, como indicar que falta un valor entero con -9999 o algún patrón de bits raro, o podría ser una convención más global, como indicar que falta un valor de coma flotante con NaN (Not a Number), un valor especial que forma parte de la especificación IEEE de coma flotante.

Ninguno de estos enfoques está exento de inconvenientes: el uso de una matriz de máscaras separada requiere la asignación de una matriz booleana adicional, lo que añade sobrecarga tanto de almacenamiento como de cálculo. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir lógica adicional (a menudo no optimizada) en la aritmética de la CPU y la GPU. Los valores especiales habituales, como NaN, no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, los distintos lenguajes y sistemas utilizan convenciones diferentes.

## Datos missing en Pandas

La forma en que Pandas maneja los valores perdidos está limitada por su dependencia de los paquete NumPy principalmente (también esta pyarrow), que no tiene una noción incorporada de valores NA para tipos de datos que no sean de coma flotante.

Pandas podría haber seguido el ejemplo de R especificando patrones de bits para cada tipo de datos individual para indicar la nulidad, pero este enfoque resulta ser bastante difícil de manejar.
Mientras que R contiene cuatro tipos de datos básicos, NumPy soporta *mucho* más que esto: por ejemplo, mientras que R tiene un único tipo entero, NumPy soporta *catorce* tipos enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la signatura y la endianidad de la codificación.
Reservar un patrón de bits específico en todos los tipos NumPy disponibles llevaría a una cantidad inmanejable de sobrecarga en operaciones especiales para varios tipos, probablemente incluso requiriendo una nueva bifurcación del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para utilizarlo como máscara reducirá significativamente el rango de valores que puede representar.

NumPy tiene soporte para arrays enmascarados, es decir, arrays que tienen un array booleano de máscara separado para marcar los datos como "buenos" o "malos".
Pandas podría haber derivado de esto, pero la sobrecarga en almacenamiento, cálculo y mantenimiento de código hace que sea una opción poco atractiva.

Con estas restricciones en mente, Pandas eligió usar centinelas para los datos perdidos, y además eligió usar valores nulos ya existentes en Python: el valor especial de coma flotante ``NaN``, el objeto Python ``None``, el valor entero ``<NA>`` y el valor datetime ``NaT`` principalmente.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Dato missing Pythonico

El primer valor centinela utilizado por Pandas es ``None``, un objeto singleton de Python que se utiliza a menudo para los datos que faltan en el código Python.
Debido a que es un objeto Python, ``None`` no se puede utilizar en cualquier array arbitrario de NumPy/Pandas, sino sólo en arrays con tipo de datos ``'object'`` (es decir, arrays de objetos Python):

In [19]:
import numpy as np
import pandas as pd
import pyarrow as pa

In [20]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy puede inferir para los contenidos del array es que son objetos Python.
Aunque este tipo de array objeto es útil para algunos propósitos, cualquier operación sobre los datos se hará a nivel Python, con mucha más sobrecarga que las típicamente rápidas operaciones vistas para arrays con tipos nativos:

In [21]:
1E6 # Son 1 millon de datos

1000000.0

In [22]:
np.arange(10, dtype=object).sum()

45

In [23]:
# Los object tardan más en procesarse
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
79.2 ms ± 4.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.8 ms ± 85.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)



El uso de objetos Python en un array también significa que si realizas agregaciones como ``sum()`` o ``min()`` a través de un array con un valor ``None``, generalmente obtendrás un error:

In [24]:
vals1.sum() # No puedo sumar entro + nonetype

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Esto refleja el hecho de que la suma entre un entero y ``None`` es indefinida.

### ``NaN``: Datos missing numéricos flotantes

Otra representación de datos que faltan, ``NaN`` (acrónimo de *Not a Number*), es diferente; se trata de un valor especial de coma flotante reconocido por todos los sistemas que utilizan la representación estándar IEEE de coma flotante:

In [ ]:
# El nan simpre se representa con numpy (np.nan)
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Fíjate en que NumPy ha elegido un tipo nativo de coma flotante para este array: esto significa que, a diferencia del array objeto de antes, este array soporta operaciones rápidas introducidas en código compilado.
Debes tener en cuenta que ``NaN`` es un poco como un virus de datos - infecta cualquier otro objeto que toca.
Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [25]:
1 + np.nan

nan

In [26]:
0 *  np.nan

nan

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a error) pero no siempre son útiles:

In [27]:
vals2.sum(), vals2.min(), vals2.max()

(np.float64(nan), np.float64(nan), np.float64(nan))

NumPy proporciona algunas agregaciones especiales que ignorarán estos valores perdidos:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2) # descartan el valor nan y hace las operaciones con los otros

Tenga en cuenta que ``NaN`` es específicamente un valor de coma flotante.

### ``<NA>``: Dato missing para todos los tipos de datos

Introducido en pandas 1.0 y mejorado en versiones posteriores, <NA> es un valor centinela que representa datos perdidos de manera unificada para todos los tipos de datos. Esto soluciona algunas limitaciones de None y NaN, que solo funcionan bien con tipos de datos específicos.

``<NA>`` es especialmente útil porque funciona con cualquier tipo de dato, ya sea numérico, booleano, categórico, o incluso fechas. Cuando se utiliza <NA>, pandas sabe cómo manejarlo de forma coherente sin generar los errores típicos que podrían ocurrir con None o NaN.

In [28]:
NA_df = pd.DataFrame({
    'int_column': pd.Series([1, 2, pd.NA, 4], dtype='Int64'),
    'float_column': pd.Series([1.2, 3.4, pd.NA, 5.6], dtype='Float64'),
    'bool_column': pd.Series([True, pd.NA, False, True], dtype='boolean'),
})

NA_df

,int_column,float_column,bool_column
0,1,1.2,True
1,2,3.4,<NA>
2,<NA>,<NA>,False
3,4,5.6,True


In [29]:
NA_df.info() # Te dice que tipo de dato son

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   int_column    3 non-null      Int64  
 1   float_column  3 non-null      Float64
 2   bool_column   3 non-null      boolean
dtypes: Float64(1), Int64(1), boolean(1)
memory usage: 208.0 bytes


### ``NaT``: Datos missing para tipos de datos de fecha y hora

``NaT`` (Not a Time) es un valor centinela utilizado para representar datos faltantes en arrays de tiempo o fecha en pandas. Similar a cómo NaN representa datos faltantes en tipos numéricos de coma flotante, NaT se utiliza exclusivamente para tipos de datos relacionados con tiempo.

In [30]:
times = pd.Series([pd.Timestamp('2023-01-01'), pd.NaT, pd.Timestamp('2023-01-03')])

times

0   2023-01-01
1          NaT
2   2023-01-03
dtype: datetime64[ns]

### PyArrow: Manejo avanzado de datos missing

Con la introducción de pandas 2.0, se añadió soporte para usar PyArrow como motor de backend para pandas. PyArrow ofrece un manejo más eficiente y avanzado de datos, especialmente para grandes volúmenes de datos, con mejor rendimiento en operaciones y menor consumo de memoria.

PyArrow también tiene su propia forma de manejar valores nulos. Cuando se utiliza PyArrow como motor, pandas utiliza la representación nativa de datos faltantes de Arrow, que es similar a ``<NA>``, pero con optimizaciones adicionales para operaciones vectorizadas y almacenamiento en memoria.

In [32]:
PyArrow_df = pd.DataFrame({
    'int_column': pd.Series([1, 2, None, 4], dtype='Int64[pyarrow]'),
    'string_column': pd.Series(['a', None, 'c', 'd'], dtype='string[pyarrow]'),
})

PyArrow_df

NameError: name 'pa' is not defined

### NaN, None, NaT y \<NA>  en Pandas

Tanto ``NaN``, ``None``,``NaT``y ``<NA>`` tienen su lugar, y Pandas está construido para manejarlos casi indistintamente, convirtiendo entre ellos cuando sea apropiado:

In [33]:
pd.Series([1, np.nan, 2, None, pd.NA, 5, pd.NaT])

0       1
1     NaN
2       2
3    None
4    <NA>
5       5
6     NaT
dtype: object

Para los tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA presentes.
Por ejemplo, si establecemos un valor en un array de enteros a ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA:

In [34]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [35]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Pero esto cambia si el dtype es `Int32` por ejemplo

In [ ]:
x = pd.Series(range(2), dtype='Int32') # Int32 con la I mayuscula acepta NA sino no
x

0    0
1    1
dtype: Int32

In [37]:
x[0] = None
x

0    <NA>
1       1
dtype: Int32

Observe que además de convertir el array de enteros a coma flotante, Pandas convierte automáticamente el valor ``None`` a un valor ``NaN``.

Aunque este tipo de magia puede parecer un poco "hacker" en comparación con el enfoque más unificado de los valores NA en lenguajes específicos de dominio como R, el enfoque centinela/casting de Pandas funciona bastante bien en la práctica y en mi experiencia sólo causa problemas en raras ocasiones.

La siguiente tabla lista las convenciones de upcasting en Pandas cuando se introducen valores NA:

|Typeclass     | Conversión al almacenar NAs | NA Valor centinela     |
|--------------|-----------------------------|------------------------|
| ``floating`` | Sin cambios                   | ``np.nan``             |
| ``object``   | Sin cambios                   | ``None`` or ``np.nan`` |
| ``integer``  | Cambia a ``float64``         | ``np.nan``             |
| ``boolean``  | Cambia to ``object``          | ``None`` or ``np.nan`` |

Tenga en cuenta que en Pandas, los datos de cadena siempre se almacenan con un dtype ``object`` por defecto, pero que puede ser modificado a ``string`` o ``category``

In [38]:
a = pd.Series(['a','b'])
a

0    a
1    b
dtype: object

In [39]:
a = pd.Series(['a','b'],dtype='string')
a

0    a
1    b
dtype: string

In [ ]:
a = pd.Series(['a','b'],dtype='category') # Este es el más óptimo, más rápido
a

0    a
1    b
dtype: category
Categories (2, object): ['a', 'b']

## Operar con valores nulos

Como hemos visto, Pandas trata NaN, None, NaT y \<NA> como esencialmente intercambiables para indicar valores perdidos o nulos.

Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Estos son:

- ``isnull()``: Generar una máscara booleana que indique los valores que faltan
- ``notnull()``: Opuesto a ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores perdidos rellenados o imputados

Concluiremos esta sección con una breve exploración y demostración de estas rutinas.

### Detección de valores nulos
Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``, ya que es imposible igualar a la nada.

Cualquiera de ellos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [41]:
data = pd.Series([1, np.nan, 'hello', None])

In [ ]:
data.isnull() # Devulve booleanos True cuando es un nulo

0    False
1     True
2    False
3     True
dtype: bool

In [43]:
data[data.isnull()]

1     NaN
3    None
dtype: object

Convertiremos a un `DataFrame` para poder tratar con mascaras que son mas visuales

In [44]:
df = pd.DataFrame(data,columns=['columna_1'])
df

,columna_1
0,1
1,NaN
2,hello
3,None


Al intentar igualar a NaN o a None no nos devuelve un error pero no nos muestro el filtro correcto

In [ ]:
df[df['columna_1'] == np.nan] # Muestra donde en la columna 1 hay un NaN

,columna_1


In [46]:
df[df['columna_1'] == None] # Muestra donde en la columna 1 hay un None

,columna_1


Esto ocurre porque los NaN son tratados como los infinitos.

$\infty \neq \infty$

In [ ]:
print(np.nan==None)
print(np.nan==np.nan)

Aunque python nos devuelva True al comparar None

In [47]:
print(None==None)

True


Hay que recordar que Pandas utiliza NumPy y por conveniencia no lo permitirá

Por ello creo el metodo ``isnull`` que es similar a ``isnan``

La diferencia principal es que ``isna`` es una forma más moderna y explícita, mientras que ``isnull`` es más tradicional y se introdujo primero.

In [ ]:
df[df['columna_1'].isna()] # .isnull funciona igual

,columna_1
1,NaN
3,None


### Eliminación de valores missing

Además del enmascaramiento utilizado anteriormente, existen los métodos ``dropna()`` (que elimina los valores NA) y ``fillna()`` (que rellena los valores NA). Para una ``Series``, el resultado es sencillo:

In [50]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [ ]:
data.dropna() # Elimina todos los valores nulos

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones.
Considera el siguiente ``DataFrame``:

In [51]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


No podemos eliminar valores individuales de un ``DataFrame``; sólo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, puede que quieras una cosa u otra, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Por defecto, ``dropna()`` eliminará todas las filas en las que *cualquier (any)* valor nulo esté presente:

In [ ]:
df.dropna() # Elimina todas las filas que tengan algún NaN

,0,1,2
1,2.0,3.0,5


Alternativamente, puede eliminar los valores nulos a lo largo de un eje diferente; ``axis=1`` o ``axis=columns`` elimina todas las columnas que contienen un valor nulo:

In [ ]:
df.dropna(axis='columns') # Al poner 'columns' elimina todas las columnas que tengan algún NaN

,2
0,2
1,5
2,6


Pero esto también elimina algunos datos buenos; quizá le interese más eliminar filas o columnas con *todos(all)* los valores NA, o con una mayoría de valores NA.
Esto se puede especificar mediante los parámetros ``how`` o ``thresh``, que permiten un control preciso del número de nulos que se permiten.

El valor predeterminado es ``how='any'``, de forma que se descartará cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo.
También puede especificar ``how='all'``, que sólo eliminará las filas/columnas que sean *todos* valores nulos:

In [54]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.dropna(axis='columns', how='all') # Elinima la columna siempre y cuando TODOS sean NaN

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Para un control más preciso, el parámetro ``thresh`` permite especificar un número mínimo de ``valores no nulos`` para la fila/columna que se desea conservar:

In [57]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.dropna(axis='rows', thresh=3) # Si tiene al menos 3 valores no nulos no eleimina la columna, si no los tiene, la elimina

,0,1,2,3
1,2.0,3.0,5,NaN


Aquí se han eliminado la primera y la última fila, porque sólo contienen dos valores no nulos.

### Rellenar valores missing

A veces, en lugar de eliminar los valores nulos, prefiere sustituirlos por un valor válido.
Este valor podría ser un número único como cero, o podría ser algún tipo de imputación o interpolación a partir de los valores buenos.
Esto se puede hacer in situ usando el método ``isnull()`` como máscara, pero como es una operación tan común Pandas proporciona el método ``fillna()``, que devuelve una copia del array con los valores nulos reemplazados.

Considera la siguiente ``Series``:

In [58]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos rellenar las entradas NA con un único valor, como cero:

In [59]:
data.fillna(0) # Rellena los valores vacíos con un 0

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Podemos especificar un forward-fill para propagar el valor anterior hacia adelante:

In [ ]:
# forward-fill (rellena hacia delante)
data.ffill()

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O podemos especificar un back-fill para propagar los siguientes valores hacia atrás:

In [ ]:
# back-fill (rellena de abajo a arriba)
data.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para ``DataFrame``s, las opciones son similares, pero también podemos especificar un ``eje`` a lo largo del cual se producen los rellenos:

In [62]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [63]:
df.ffill(axis = 'columns')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Tenga en cuenta que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.

<!--NAVIGATION-->
< [Operaciones en Pandas](3-Operaciones_en_Pandas.ipynb) | [Concatenado](5-Concatenado.ipynb) >
